In [1]:
import os
import csv
import numpy as np
import cv2
from matplotlib import pyplot as plt
from matplotlib import cm
from PIL import Image

In [2]:
dataset_folder     = 'D:/DIODE'
data_list_folder   = 'data_list'
train_indoors_file = 'train_indoors.csv'
val_indoors_file   = 'val_indoors.csv'
rgb_folder         = 'rgb'
depth_folder       = 'depth'

resize_to = (128, 128)#(224, 224)

number_of_images_to_plot = 0#2

DATALIST_DIR            = os.path.join(dataset_folder, data_list_folder)
TRAIN_INDOORS_FILE_PATH = os.path.join(DATALIST_DIR, train_indoors_file)
VAL_INDOORS_FILE_PATH   = os.path.join(DATALIST_DIR, val_indoors_file)
RGB_DIR                 = os.path.join(dataset_folder, rgb_folder)
DEPTH_DIR               = os.path.join(dataset_folder, depth_folder)

In [3]:
def get_list_of_holes(depth_mask):
    result = []
    for i in range(depth_mask.shape[0]):
        for j in range(depth_mask.shape[1]):
            if depth_mask[i, j] == 0:
                result.append((i, j))
    return result

def find_neighbors_average(depth, depth_mask, pixel):
    height = depth.shape[0]
    width  = depth.shape[1]
    result = None
    sum    = 0
    count  = 0
    list_of_neighbors_difs = [(-1, -1), (-1, 0), (-1, 1), (0, -1), (0, 1), (1, -1), (1, 0), (1, 1)]
    for neighbor_dif in list_of_neighbors_difs:
        neighbor = (pixel[0] + neighbor_dif[0], pixel[1] + neighbor_dif[1])
        if 0 <= neighbor[0] < height and 0 <= neighbor[1] < width:
            if depth_mask[neighbor[0], neighbor[1]] == 1:
                sum += depth[neighbor[0], neighbor[1]]
                count += 1
    if 0 < count:
        result = sum / count
    return result

def fix_depth_image_using_mask(depth, depth_mask):
    list_of_holes = get_list_of_holes(depth_mask=depth_mask)
    while 0 < len(list_of_holes):
        for hole in list_of_holes:
            neighbors_average = find_neighbors_average(depth=depth, depth_mask=depth_mask, pixel=hole)
            if neighbors_average is not None:
                depth[hole[0], hole[1]]      = neighbors_average
                depth_mask[hole[0], hole[1]] = 1
        list_of_holes = get_list_of_holes(depth_mask=depth_mask)
    return depth

def sort_files_to_folders(csv_file_path, first_image_number=0):
    i = first_image_number
    j = 0
    with open(csv_file_path) as csv_file:
        csv_reader = csv.reader(csv_file, delimiter=',')
        fig, axes = plt.subplots(nrows=number_of_images_to_plot, ncols=4, figsize=(16,16), subplot_kw={'aspect': 1})
        plt.tight_layout()
        for row in csv_reader:
            rgb         = Image.open(os.path.join(dataset_folder, row[0][2:]))
            depth       = np.load(os.path.join(dataset_folder, row[1][2:])).squeeze()
            depth_mask  = np.load(os.path.join(dataset_folder, row[2][2:])).squeeze()
            depth_fixed = fix_depth_image_using_mask(depth=depth.copy(), depth_mask=depth_mask.copy())
            if j < number_of_images_to_plot:
                axes[i, 0].imshow(rgb)
                axes[i, 0].set_title('RGB')
                axes[i, 1].imshow(depth_fixed)
                axes[i, 1].set_title('Depth\nNo holes')
                axes[i, 2].imshow(depth)
                axes[i, 2].set_title('Depth\nWith holes')
                axes[i, 3].imshow(depth_mask)
                axes[i, 3].set_title('Holes mask')
            depth_fixed = depth_fixed - np.min(depth_fixed)
            depth_fixed = depth_fixed / np.max(depth_fixed)
            rgb         = rgb.resize(resize_to) 
            depth_fixed = cv2.resize(depth_fixed, dsize=resize_to, interpolation=cv2.INTER_NEAREST) * 255
            rgb.save(RGB_DIR + '/' + str(i) + '.png', "PNG")
            cv2.imwrite(DEPTH_DIR + '/' + str(i) + '.png', depth_fixed)
            if i % 50 == 0:
                print("Finished " + str(i))
#             if (j + 1) == number_of_images_to_plot:
#                 break
            i += 1
            j += 1
    return i

In [4]:
if not os.path.exists(RGB_DIR):
    os.makedirs(RGB_DIR)
if not os.path.exists(DEPTH_DIR):
    os.makedirs(DEPTH_DIR)

current_image_number = 0
current_image_number = sort_files_to_folders(csv_file_path=TRAIN_INDOORS_FILE_PATH, first_image_number=current_image_number)
current_image_number = sort_files_to_folders(csv_file_path=VAL_INDOORS_FILE_PATH, first_image_number=current_image_number)

Finished 0
Finished 50
Finished 100
Finished 150
Finished 200
Finished 250
Finished 300
Finished 350
Finished 400
Finished 450
Finished 500
Finished 550
Finished 600
Finished 650
Finished 700
Finished 750
Finished 800
Finished 850
Finished 900
Finished 950
Finished 1000
Finished 1050
Finished 1100
Finished 1150
Finished 1200
Finished 1250
Finished 1300
Finished 1350
Finished 1400
Finished 1450
Finished 1500
Finished 1550
Finished 1600
Finished 1650
Finished 1700
Finished 1750
Finished 1800
Finished 1850
Finished 1900
Finished 1950
Finished 2000
Finished 2050
Finished 2100
Finished 2150
Finished 2200
Finished 2250
Finished 2300
Finished 2350
Finished 2400
Finished 2450
Finished 2500
Finished 2550
Finished 2600
Finished 2650
Finished 2700
Finished 2750
Finished 2800
Finished 2850
Finished 2900
Finished 2950
Finished 3000
Finished 3050
Finished 3100
Finished 3150
Finished 3200
Finished 3250
Finished 3300
Finished 3350
Finished 3400
Finished 3450
Finished 3500
Finished 3550
Finished 3600
F

<Figure size 1152x1152 with 0 Axes>

<Figure size 1152x1152 with 0 Axes>